This Notebook is a kaggle tutorial for Japanese kaggle beginners writen in Japanese.

# 4. 勾配ブースティングが最強？！ いろいろな機械学習アルゴリズムを使ってみよう

これまでは機械学習アルゴリズムとして、ロジスティック回帰を採用していました。

この[Notebook](https://www.kaggle.com/sishihara/upura-kaggle-tutorial-04-lightgbm)では、いろいろな機械学習アルゴリズムを使ってみましょう。これまでロジスティック回帰を使っていた部分を差し替えて学習・予測を実行してみたいと思います。

ロジスティック回帰の実装に利用していたsklearnというパッケージは入出力のインタフェースが統一されており、手軽に機械学習アルゴリズムを変更できます。実際にいくつか試してみましょう。

また最近のKaggleのコンペティションで上位陣が利用している機械学習アルゴリズムとしては、勾配ブースティングやニューラルネットワークが挙げられます。これらはロジスティック回帰に比べて表現力が高く、高性能に予測できる可能性を秘めています。特に上位陣での採用率が高いのは「LightGBM」という勾配ブースティングのパッケージです。sklearnと同様のインターフェイスも用意されていますが、ここでは[Python-package Introduction](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)に記載の方式で実装します。

In [ ]:
# 特徴量の準備

import numpy as np
import pandas as pd

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
data.head()

In [ ]:
delete_columns = ['Name', 'PassengerId','Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [ ]:
X_train.head()

特徴量の準備が完了しました。

# sklearn
まずはsklearn内で機械学習アルゴリズムを変更していきましょう。これまではロジスティック回帰を使ってきました。

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', solver="sag", random_state=0)

sklearnでは、clfで宣言するモデルを切り替えるだけで機械学習アルゴリズムを差し替えられます。例えば、[ランダムフォレスト](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)と呼ばれる機械学習アルゴリズムを使ってみましょう。

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

あとはロジスティック回帰の場合と同様に学習・予測が実行可能です。

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred[:10]

In [ ]:
sub = gender_submission

In [ ]:
sub['Survived'] = list(map(int, y_pred))
sub.to_csv("submission_randomforest.csv", index=False)
sub.head()

ランダムフォレストでの予測結果を提出してみると、私の環境では0.77033というロジスティック回帰の時よりも格段に良いスコアが出ました。

[sklearnには非常に多くの種類の機械学習アルゴリズムが実装されている](https://scikit-learn.org/stable/supervised_learning.html)ので、ぜひいろいろな機械学習アルゴリズムを試してみてください。

# LightGBM
続いて、LightGBMを使います。sklearnとの差異もあり、いくつか下準備が必要です。

1. 学習用・検証用にデータセットを分割する
2. カテゴリ変数をリスト形式で宣言する

LightGBMは大量の決定木を作成しながら学習を進めます。そのため、学習に利用したデータセットなどにのみ過剰に適合してしまい、本来の目的である未知の値に対する性能が劣化してしまう「過学習」という現象に陥りがちです。そこで学習に利用しない検証用のデータに対する性能を見ながら学習を打ち切る「early stopping」を利用するのが一般的となっています。

ここでは、X_trainをX_train（学習用）とX_valid（検証用）に分割します。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

[LightGBMでは、カテゴリ変数に対して特別な処理](https://tebasakisan.hatenadiary.com/entry/2019/01/27/222102)を自動的に実行してくれます。次のように、何をカテゴリ変数として扱ってほしいか明示的にLightGBMに教えてあげましょう。

In [ ]:
categorical_features = ['Embarked', 'Pclass', 'Sex']

下準備も終わったところで、LightGBMで学習・予測を実施します。

In [ ]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {
    'objective': 'binary'
}

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(10)]
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]

今回のLightGBMの設定では、出力結果は1になる予測値になります。今回はしきい値を決め打って、0.5を上回っていれば1と予測したと見なして、提出してみます。

In [ ]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

In [ ]:
sub['Survived'] = y_pred
sub.to_csv("submission_lightgbm.csv", index=False)

sub.head()

LightGBMでの予測結果を提出してみると、私の環境では0.75598というスコアが出ました。ランダムフォレスト同様、ロジスティック回帰の時のスコアよりも向上しているのが分かります。

このように利用する機械学習アルゴリズム次第で、Kaggleのスコアを向上させることが可能です。